In [1]:
from cropengine.weather import GEEWeatherDataProvider

In [2]:
import ee
import geemap
ee.Initialize()

In [3]:
Map = geemap.Map(basemap='SATELLITE')
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', transp…

In [5]:
region = Map.draw_last_feature

In [6]:
# climdata = GEEWeatherDataProvider(latitude=52.06912171608079, longitude=10.881185799808796, start_date='2020-01-01', end_date='2022-12-31', source='era5_land')
climdata_region = GEEWeatherDataProvider(geometry=region, reducer='mean', start_date='2020-01-01', end_date='2022-12-31', source='era5_land')

In [12]:
climdata.get_data()

,date,TMAX,TMIN,VAP,WIND,RAIN,IRRAD,SNOWDEPTH
0,2020-01-01,2.000,-2.942,0.563,1.518,0.003,2317.176,-0.0
1,2020-01-02,2.868,-3.834,0.495,1.749,0.004,3415.148,-0.0
2,2020-01-03,7.080,2.476,0.795,3.869,2.618,680.796,-0.0
3,2020-01-04,5.419,1.830,0.714,5.193,3.873,904.040,-0.0
4,2020-01-05,4.349,-0.084,0.649,2.151,0.073,2467.152,-0.0
...,...,...,...,...,...,...,...,...
1090,2022-12-26,10.498,4.149,0.964,4.157,2.074,2031.788,-0.0
1091,2022-12-27,5.051,2.605,0.636,4.095,0.288,2425.172,-0.0
1092,2022-12-28,8.164,3.552,0.783,4.921,2.122,358.600,-0.0
1093,2022-12-29,10.771,7.258,0.916,5.232,2.154,1689.768,-0.0


In [ ]:
# climdata_region.save_weather_excel('data.xlsx')

File saved successfully to data.xlsx
